In [1]:
from __future__ import annotations

import geopandas as gpd
from osmnx import features

from cityseer.tools import io, graphs
from cityseer.metrics import networks, layers

This example uses the `momepy` `bubenec.gpkg` GeoPandas GeoDataFrame as an example. Please download this file and place it in a locally accessible file. Update the filepaths in the next code block accordingly.


In [2]:
from pathlib import Path

repo_path = Path.cwd()
if str(repo_path).endswith("notebooks"):
    repo_path = Path.cwd() / ".."
if not str(repo_path.resolve()).endswith("cityseer-examples"):
    raise ValueError(
        "Please check your notebook working directory relative to your project and data paths."
    )

bubenec_path = Path(repo_path / "temp/bubenec.gpkg")
print("data path:", bubenec_path)
print("path exists:", bubenec_path.exists())

df_streets: gpd.GeoDataFrame = gpd.read_file(bubenec_path, layer="streets")  # type: ignore
nx_momepy = io.nx_from_generic_geopandas(df_streets)

ValueError: Please check your notebook working directory relative to your project and data paths.

At this point the input GeoPandas file has been converted to a `networkX` `MultiGraph` with `geom` attributes for the edges. This graph can now be fed to `cityseer` `graph` module methods if wanted.
For running optimised network centrality or landuse accessibilities, use the `io.network_structure_from_nx` method to prepare the necessary structures.


In [ ]:
nodes_gdf, edges_gdf, network_structure = io.network_structure_from_nx(
    nx_momepy, crs=df_streets.crs.to_epsg()
)

The following block downloads some OSM data which will be used for demonstrating some landuse accessibility methods.


In [ ]:
nodes_gdf.head()

In [ ]:
edges_gdf.head(3)

In [ ]:
data_gdf = features.features_from_point(
    (50.1029248, 14.4029967), tags={"amenity": ["pub", "restaurant"]}, dist=400
)
data_gdf.to_crs(3857, inplace=True)
data_gdf = data_gdf.loc["node"]
data_gdf = data_gdf.reset_index(level=0, drop=True)
data_gdf.index = data_gdf.index.astype(str)
data_gdf = data_gdf[["amenity", "geometry"]]
print(data_gdf.head())

In [ ]:
# compute metrics
nodes_gdf = networks.node_centrality_shortest(
    network_structure, nodes_gdf, [250, 500, 1000]
)
nodes_gdf, data_gdf = layers.compute_accessibilities(
    data_gdf,
    landuse_column_label="amenity",
    accessibility_keys=["restaurant"],
    nodes_gdf=nodes_gdf,
    network_structure=network_structure,
    distances=[100, 200, 400],
)
nodes_gdf.head()

In [ ]:
# if using dual network
nx_dual = graphs.nx_to_dual(nx_momepy)
nodes_gdf_dual, _edges_gdf_dual, network_structure_dual = io.network_structure_from_nx(
    nx_dual, df_streets.crs.to_epsg()
)
nodes_gdf_dual = networks.node_centrality_simplest(
    network_structure_dual, nodes_gdf_dual, [500]
)
nodes_gdf_dual.head()